In [1]:
import pandas as pd
import json
import os
from glob import glob

In [2]:
experiment = 'cifar10'
files = sorted(glob(f"./stats/{experiment}/*.json"))
experiment = 'alexnet_energy'
files = files + sorted(glob(f"./stats/{experiment}/*.json"))
# experiment = 'alexnet_val_sweep'
# files = sorted(glob(f"./stats/{experiment}/*.json"))

In [3]:
initial_r2 = []
final_r2 = []
test_acc = []
training_steps = []
datasets = []
criterion = []
val_proportion = []
for file in files:
    j = json.load(open(file,"r"))
    if j["val_proportion"] in [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]:
# #         print(file)
#         os.system(f"rm {file}")
        continue
    #     if j['dataset'] == 'cifar10':
    initial_r2.append(j['initial_r2'])
    final_r2.append(j['final_r2'])
    test_acc.append(j['test_acc'])
    training_steps.append(j['training_steps'])
    datasets.append(j['dataset'])
    criterion.append(j['stopping_criterion'])
    val_proportion.append(j['val_proportion'])
    
df_exp = pd.DataFrame(zip(datasets, initial_r2, final_r2, test_acc, training_steps, criterion, val_proportion), 
                       columns=['dataset', 'initial_mlh', 'final_mlh', 'test_acc', 'training_steps', 'stopping_criterion', "val_proportion"])
# df_exp1 = df_exp1.loc[df_exp1.dataset == 'mnist']
df_exp

,dataset,initial_mlh,final_mlh,test_acc,training_steps,stopping_criterion,val_proportion
0,cifar10,0.065830,0.001315,0.627872,2548,energy,None
1,cifar10,0.065864,0.002880,0.637253,2940,energy,None
2,cifar10,0.065837,0.001526,0.637489,3234,energy,None
3,cifar10,0.066675,0.001534,0.639310,3234,energy,None
4,cifar10,0.065048,0.002434,0.640315,2450,energy,None
5,cifar10,0.066612,0.002558,0.630934,3136,energy,None
6,cifar10,0.066286,0.001347,0.641182,2744,energy,None
7,cifar10,0.065216,0.001922,0.635857,3234,energy,None
8,cifar10,0.065996,0.000982,0.628699,2744,energy,None
9,cifar10,0.065633,0.001684,0.628573,2842,energy,None


In [4]:
import numpy as np
df_exp["val_proportion"].replace(np.nan, 0.0).value_counts()

0.0    19
Name: val_proportion, dtype: int64

In [5]:
import plotly.express as px
# df_exp>
def convert_to_exponent(x):
    if len(str(x)) > 4:
        return "{:.2e}".format(x)
    return str(x)

df_temp = df_exp.replace(np.nan, 0.0)
df_temp["val_proportion"] = df_temp["val_proportion"].apply(convert_to_exponent)
df_temp.rename({"stopping_criterion": "Stopping Criterion"}, inplace=True, axis=1)
df_temp.replace({"loss/val": "Validation Loss", "energy": "MLH"}, inplace=True)
# df_temp = df_temp.sort_values("val_proportion", key=lambda y: sorted(y, key=lambda x: float(x)))
category_orders = sorted(list(df_temp.val_proportion.unique()), key=lambda x: float(x))
fig = px.box(df_temp, x="val_proportion", y="test_acc", color="Stopping Criterion", category_orders={"val_proportion": category_orders})
fig.update_layout(xaxis={'tickformat': 'e', 'showexponent': 'all'})
# fig.update_layout(yaxis={'tickformat': 'e', 'showexponent': 'all'})
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.21,
    xanchor="left",
    x=0.75,bordercolor="Black", borderwidth=1
), width=700, height=500)




# fig.update_layout({
# "plot_bgcolor": "rgba(0, 0, 0, 0)",
# "paper_bgcolor": "rgba(255, 255, 255, 255)",
# })

fig.update_layout(title_text="",
        paper_bgcolor='rgba(255,255,255,1)',\
    plot_bgcolor='rgba(255,255,255,1)',)

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True,ticks='outside', showgrid=True, gridcolor="LightGray")
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks='outside', showgrid=True, gridcolor="LightGray")

fig.update_layout(xaxis_title="Validation Split Proportion", yaxis_title="Test Accuracy", font=dict(family='Times New Roman', size=14, color='Black'))
fig.show()

In [18]:
fig.write_image("./val_prop.png")

In [7]:
# out = scipy.stats.ttest_ind(
#     df_exp[df_exp.stopping_criterion == "energy"].test_acc.values, 
#     df_exp[df_exp.stopping_criterion != "energy"].test_acc.values, 
#     equal_var=False
# )
# out.statistic, out.pvalue

In [8]:
temp = df_exp.replace(np.nan, 0.0)
temp = temp[['val_proportion', 'test_acc']]
temp['val_proportion'] = temp['val_proportion'].astype(str)
mean = temp.groupby("val_proportion").mean().reset_index()
std = temp.groupby("val_proportion").std().reset_index()
temp = pd.merge(mean, std, on="val_proportion")
temp['val_proportion'] = temp['val_proportion'].astype(float).apply(lambda x: round(x, 4))
temp = temp.rename({
    "val_proportion": "Validation Proportion",
    "test_acc_x": "Mean Test Accuracy",
    "test_acc_y": "Standard Deviation Test Accuracy",
}, axis=1)
temp['Early Stopping Criterion'] = ['MLH'] + ['Validation Accuracy'] * (len(temp)-1) 
temp = temp[['Early Stopping Criterion', 'Validation Proportion', 'Mean Test Accuracy', 'Standard Deviation Test Accuracy']]
temp

,Early Stopping Criterion,Validation Proportion,Mean Test Accuracy,Standard Deviation Test Accuracy
0,MLH,0.0000,0.589683,0.015108
1,Validation Accuracy,0.0001,0.539487,0.034870
2,Validation Accuracy,0.0004,0.561775,0.027827
3,Validation Accuracy,0.0016,0.574701,0.021253
4,Validation Accuracy,0.0251,0.580396,0.018062
5,Validation Accuracy,0.1000,0.579036,0.021034
6,Validation Accuracy,0.2000,0.575759,0.015814
7,Validation Accuracy,0.3000,0.563714,0.019636
8,Validation Accuracy,0.4000,0.555380,0.017576


In [9]:
print(temp.to_markdown(index=False))

| Early Stopping Criterion   |   Validation Proportion |   Mean Test Accuracy |   Standard Deviation Test Accuracy |
|:---------------------------|------------------------:|---------------------:|-----------------------------------:|
| MLH                        |                  0      |             0.589683 |                          0.0151077 |
| Validation Accuracy        |                  0.0001 |             0.539487 |                          0.0348697 |
| Validation Accuracy        |                  0.0004 |             0.561775 |                          0.0278269 |
| Validation Accuracy        |                  0.0016 |             0.574701 |                          0.0212528 |
| Validation Accuracy        |                  0.0251 |             0.580396 |                          0.0180618 |
| Validation Accuracy        |                  0.1    |             0.579036 |                          0.0210344 |
| Validation Accuracy        |                  0.2    |        

In [10]:
files = glob("./converted/*.csv")
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=3, shared_xaxes=True, x_title="Training Steps")
for f in files:
    df = pd.read_csv(f)

    acc_train = df.loc[df.metric == "acc/train"]
    fig.add_trace(
        go.Scatter(x=acc_train.step, y=acc_train.value),
        row=1, col=1
    )

    acc_val = df.loc[df.metric == "acc/val"]
    fig.add_trace(
        go.Scatter(x=acc_val.step, y=acc_val.value),
        row=1, col=2
    )
    
    mlh = df.loc[df.metric == "mlh/train"]
    fig.add_trace(
        go.Scatter(x=mlh.step, y=mlh.value),
        row=1, col=3,
        
    )

fig.update_layout(showlegend=False, width=1500)
fig.update_xaxes(
    range=[0, 6000],
    tickmode='array',
    tickvals=[0, 2000, 4000, 6000],
    ticktext=["0", "2K", "4K", "6K"]
)

fig.update_yaxes(title_text="Training Accuracy", row=1, col=1)
fig.update_yaxes(title_text="Validation Accuracy", row=1, col=2)
fig.update_yaxes(title_text="MLH", row=1, col=3)
fig.update_yaxes(title_standoff=1)

fig.update_layout(
    font=dict(
        family="Times New Roman",
        size=29,
        color="Black",
    ), 
    autosize=True,
    # coloraxis={"colorscale":"inferno"}
)
fig.update_annotations(font_size=29)

fig.update_layout(template='none', font={'size': 20, "family":'Times New Roman', "color":"#000000"}, title=None, legend_borderwidth=1)
fig.update_xaxes(mirror=True, ticks="outside", showline=True, automargin=True, showgrid=True)
fig.update_yaxes(mirror=True, ticks="outside", showline=True, automargin=True, showgrid=True)

fig.show()